In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

# 데이터프레임을 출력했을때 더 많은 열이 보이도록 함
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 300)
# movieId를 기준으로 ratings 와 movies 를 결합함
movie_ratings = pd.merge(ratings, movies, on='movieId')
print(movie_ratings)

# 유저 기반 협업 필터링

In [ ]:
# user별로 영화에 부여한 rating 값을 볼 수 있도록 pivot table 사용
title_user = movie_ratings.pivot_table('rating', index='userId', columns='title')

# 평점을 부여안한 영화는 그냥 0이라고 부여
title_user = title_user.fillna(0)
print(title_user)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 유저 1~610 번과 유저 1~610 번 간의 코사인 유사도를 구함
user_based_collab = cosine_similarity(title_user, title_user)
print(user_based_collab)

In [ ]:
# 위는 그냥 numpy 행렬이니까, 이를 데이터프레임으로 변환
user_based_collab = pd.DataFrame(user_based_collab, index=title_user.index, columns=title_user.index)
print(user_based_collab)

In [ ]:
# 1번 유저와 비슷한 유저를 내림차순으로 정렬한 후에, 상위 10개만 뽑음
print(user_based_collab[1].sort_values(ascending=False)[:10])

In [ ]:
# 1번 유저와 가장 비슷한 266번 유저를 뽑고,
user = user_based_collab[1].sort_values(ascending=False)[:10].index[1]
# 266번 유저가 좋아했던 영화를 평점 내림차순으로 출력
result = title_user.query(f"userId == {user}").sort_values(ascending=False, by=user, axis=1)
print(result)

In [ ]:
# 만약 해당 유저가 아직 보지 않은 영화에 대해서, 평점을 예측하고자 한다면?
# (어떤 유저와 비슷한 정도 * 그 유저가 영화에 대해 부여한 평점) 을 더해서 (유저와 비슷한 정도의 합)으로 나눠보면 됨!
# index_list 는 비슷한 유저의 id 값 리스트 / weight_list 는 비슷한 유저와의 유사도 리스트
user_index_list = user_based_collab[1].sort_values(ascending=False)[:10].index.tolist()
user_weight_list = user_based_collab[1].sort_values(ascending=False)[:10].tolist()

In [ ]:
# 1번 유저가 다크나이트를 보고 어떤 평점을 부여할지 예측
movie_title = 'Dark Knight, The (2008)'
weighted_sum = []
weighted_user = []
for i in range(1, 10):
    # 해당 영화를 보고 평점을 부여한 사람들의 유사도와 평점만 추가 (즉, 0이 아닌 경우에만 계산에 활용)
    if int(title_user[movie_title][user_index_list[i]]) is not 0:
        # 평점 * 유사도 추가
        weighted_sum.append(title_user[movie_title][user_index_list[i]] * user_weight_list[i])
        # 유사도 추가
        weighted_user.append(user_weight_list[i])

print(weighted_sum)
print(weighted_user)
# 총 평점*유사도 / 총 유사도를 토대로 평점 예측
print(sum(weighted_sum)/sum(weighted_user))

# 아이템 기반 협업 필터링

In [ ]:
# 이번에는 Index에 title이 들어감!
user_title = movie_ratings.pivot_table('rating', index='title', columns='userId')

user_title = user_title.fillna(0)
print(user_title)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

item_based_collab = cosine_similarity(user_title, user_title)
print(item_based_collab)

In [ ]:
item_based_collab = pd.DataFrame(item_based_collab, index=user_title.index, columns=user_title.index)

In [ ]:
print(item_based_collab)

In [ ]:
# 다크나이트와 비슷하게 유저들로부터 평점을 부여받은 영화들은?
print(item_based_collab['Dark Knight, The (2008)'].sort_values(ascending=False)[:10])

# 잠재 요인 협업 필터링

In [ ]:
# surprise 라는 모듈을 사용합니다
!pip install scikit-surprise

In [ ]:
from surprise import SVD, Dataset, accuracy, Reader
from surprise.model_selection import train_test_split

In [ ]:
ratings = pd.read_csv('ratings.csv')
# 평점의 최소값, 최대값 범위를 지정해줍니다
reader = Reader(rating_scale=(1.0, 5.0))
# 불러온 csv 파일을 surprise 모듈을 통해 분석할 수 있도록 불러옵니다
data = Dataset.load_from_df(df=ratings[['userId', 'movieId', 'rating']], reader=reader)

In [ ]:
# 에러 계산을 위해, 테스트/트레인 셋 나눠줍니다 (3:1로 나눴습니다!)
train, test = train_test_split(data, test_size=0.25, shuffle=True, random_state=23)

# n_factors 는 잠재요인의 크기를 말하는 것
algo = SVD(n_factors=50, n_epochs=20, random_state=23)
# 가장 현재 데이터를 잘 설명할 수 있도록 (최적화하면서) 행렬을 분해함
algo.fit(trainset=train)

In [ ]:
# 에러 계산
pred = algo.test(testset=test)
accuracy.rmse(predictions=pred)

In [ ]:
# 10번 유저가 200번 영화를 보고 부여할 평점을 예측함
pred = algo.predict('10', '200')
# print 했을때 나오는 r_ui 는 실제 평점 / est 는 예측 평점
# 실제 평점이 없다면 none 이라고 표시됩니다!
print(pred)

In [ ]:
preds = []
for i in ratings['movieId'].unique():
    # 10번 유저에 대해서, 10번 유저가 부여할 모든 영화에 대한 평점을 예측
    pred = algo.predict('10', i)
    # iid 는 아이템아이디, est 는 예측한 평점값
    preds.append((pred.est, pred.iid))

In [ ]:
# 10번 유저가 모든 영화에 대해 부여할 평점을 내림차순으로 정렬
preds.sort(reverse=True)
print(preds)

In [ ]:
# 위에서 확인한 영화 id를 통해서, 10번 유저가 가장 좋아하는 영화의 이름을 찾아서 출력
movies = pd.read_csv('movies.csv')
print(movies[movies['movieId']==318])